In [1]:
import os
import shutil
import requests
import gzip
import sys
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
from statsmodels.stats.rates import test_poisson, confint_poisson

# list of GPCR genes

In [13]:
receptors = requests.get('https://gpcrdb.org/services/receptorlist/').json()
print(len(receptors))
human_receptors = [receptor for receptor in receptors 
    if (receptor['species'] == 'Homo sapiens')]
print(len(human_receptors))
human_nonolf_receptors = [receptor for receptor in human_receptors 
    if (receptor['receptor_family'] not in ['Olfactory'])]
print(len(human_nonolf_receptors))
human_receptors = pd.DataFrame(human_nonolf_receptors)
human_receptors = human_receptors[~human_receptors.entry_name.duplicated()]
print(human_receptors.shape[0])

2056
423
397
397


In [17]:
human_receptors_ref = pd.read_csv('../data/labels/gene_families/gpcr_genes_human_gpcrdb.tsv',sep='\t')

In [25]:
human_receptors[human_receptors.entry_name.isin(pseudogenes)]

,entry_name,name,accession,receptor_class,receptor_family,ligand_type,subfamily,endogenous_ligands,species
79,npy6r_human,y<sub>6</sub> receptor,Q99463,Class A (Rhodopsin),Neuropeptide Y receptors,Peptide receptors,y<sub>6</sub> receptor,[],Homo sapiens
222,gpr33_human,<i>GPR33</i>,Q49SQ1,Class A (Rhodopsin),Class A orphans,Orphan receptors,<i>GPR33</i>,[],Homo sapiens
282,taar3_human,<i>TAAR3</i>,Q9P1P4,Class A (Rhodopsin),Class A orphans,Orphan receptors,<i>TAAR3</i>,[{'name': 'isoamylamine'}],Homo sapiens
286,taar9_human,<i>TAAR9</i>,Q96RI9,Class A (Rhodopsin),Class A orphans,Orphan receptors,<i>TAAR9</i>,[],Homo sapiens
310,agre4_human,ADGRE4P,Q86SQ3,Class B2 (Adhesion),ADGRE,Adhesion receptors,ADGRE4P,[],Homo sapiens
388,t2r45_human,<i>TAS2R45</i>,P59539,Class T (Taste 2),Taste 2 receptors,Sensory receptors,<i>TAS2R45</i>,[],Homo sapiens


In [24]:
pseudogenes = [name for name in human_receptors.entry_name if name not in human_receptors_ref.entry_name.values]

0      5ht1a_human
1      5ht1b_human
2      5ht1d_human
3      5ht1e_human
4      5ht1f_human
          ...     
392    gp107_human
393    g137a_human
394    tpra1_human
395    gp143_human
396    gp157_human
Name: entry_name, Length: 397, dtype: object

In [ ]:
# Get all human receptors from GPCRdb
receptors = requests.get('https://gpcrdb.org/services/receptorlist/').json()
human_receptors = [receptor for receptor in receptors \
    if (receptor['species'] == 'Homo sapiens') & \
        (receptor['receptor_family'] not in ['Olfactory receptors'])
        ]
human_receptors = pd.DataFrame(human_receptors)
human_receptors['is_orphan'] = human_receptors.endogenous_ligands.map(len) == 0
human_receptors = human_receptors[
    ['entry_name',
    'name',
    'accession',
    'receptor_class',
    'receptor_family',
    'ligand_type',
    'subfamily',
    'is_orphan'
    ]]

# Annotate with genes and coordinates using Uniprot Proteins API
def get_genes(uniprot_accession):
    requestURL = f"https://www.ebi.ac.uk/proteins/api/proteins/{uniprot_accession}"

    r = requests.get(requestURL, headers={ "Accept" : "application/json"})

    if not r.ok:
        r.raise_for_status()
        sys.exit()

    responseBody = r.json()
    return responseBody['gene']


genes = []
for uniprot_accession in tqdm(human_receptors.accession):
    genes.append(get_genes(uniprot_accession))

# Annotate with gene names and synonyms
human_receptors['gene'] = [gene[0]['name']['value'] for gene in genes]

gene_names = []
for gene in genes:
    primary_name = gene[0]['name']['value']
    if 'synonyms' in gene[0].keys():
        synonyms = [synonym['value'] for synonym in gene[0]['synonyms']]
        gene_names.append([primary_name] + synonyms)
    else:
        gene_names.append(primary_name)
human_receptors['gene_names'] = gene_names

human_receptors = human_receptors[~human_receptors.entry_name.duplicated()]

human_receptors.to_csv('../data/gpcr_genes_human_gpcrdb.tsv','\t')


# Constraint from gnomAD

In [66]:
gnomad_constraint_path = "../data/all_genes_constraint_gnomad.tsv"
r = requests.get("https://storage.googleapis.com/gcp-public-data--gnomad/release/2.1.1/constraint/gnomad.v2.1.1.lof_metrics.by_gene.txt.bgz",stream=True)
if r.status_code == 200:
    with open(gnomad_constraint_path,'wb') as f:
        r.raw.decode_content = True  # just in case transport encoding was applied
        gzip_file = gzip.GzipFile(fileobj=r.raw)
        shutil.copyfileobj(gzip_file, f)

In [55]:
# Get constraint for all gene regions in gnomad from gnomad summary statistics
gnomad_constraint_path = "../data/all_genes_constraint_gnomad.tsv"
if not os.path.exists(gnomad_constraint_path):
    r = requests.get("https://storage.googleapis.com/gcp-public-data--gnomad/release/2.1.1/constraint/gnomad.v2.1.1.lof_metrics.by_gene.txt.bgz",stream=True)
    if r.status_code == 200:
        with open(gnomad_constraint_path,'wb') as f:
            r.raw.decode_content = True  # just in case transport encoding was applied
            gzip_file = gzip.GzipFile(fileobj=r.raw)
            shutil.copyfileobj(gzip_file, f)


def clean_names(names):
    try:
        return eval(names)
    except:
        return names

# Match to genes in gnomAD constraint calculations - check that none are missed
gpcr_human_genes = pd.read_csv('../data/gpcr_genes_human_gpcrdb.tsv',sep='\t',converters={'gene_names': clean_names})
gnomad_constraint = pd.read_csv(gnomad_constraint_path,sep='\t')

gene_name_lookup = gpcr_human_genes[['gene','gene_names']].explode(column='gene_names')
problematic_mappings = pd.DataFrame([
    ('ADRA1D', 'ADRA1A'),
    ('CCRL2', 'CCR6'),
    ('ACKR2','CCR10'),
    ('S1PR3','C9orf47'),
    ('PTGER4' ,'PTGER2'),
    ('FZD9' ,'FZD3')],columns =['gene','gene_names'])
gene_name_lookup = problematic_mappings.merge(gene_name_lookup,how='outer',indicator=True)
gene_name_lookup = gene_name_lookup[gene_name_lookup._merge != 'both'].drop(columns=['_merge'])
gnomad_constraint = gene_name_lookup.merge(gnomad_constraint,left_on='gene_names',right_on='gene',suffixes=['','_gnomad'])
gnomad_constraint = gnomad_constraint[gnomad_constraint.transcript != 'ENST00000528054'] # remove non-canonical transcripts

# Select columns needed for analysis
gnomad_constraint = gnomad_constraint[[
    'gene','gene_gnomad','transcript',
    'obs_lof','exp_lof',
    'obs_mis','exp_mis',
    'obs_mis_pphen','exp_mis_pphen',
    'obs_syn','exp_syn',
    'pLI'
]]
gnomad_constraint['obs_mis_non_pphen'] = gnomad_constraint.obs_mis - gnomad_constraint.obs_mis_pphen
gnomad_constraint['exp_mis_non_pphen'] = gnomad_constraint.exp_mis - gnomad_constraint.exp_mis_pphen

def poisson_test(obs, exp):
    test = test_poisson(
        count = obs, 
        nobs=1,
        value=exp,
        method="exact-c",
        alternative = "smaller"
    )
    ci = confint_poisson(
        count = obs, 
        exposure=exp,
        method="exact-c"
    )
    return [obs/exp, ci[0], ci[1], np.log(test.pvalue)]

for impact in ['syn','mis_non_pphen','mis_pphen','lof']:
    gnomad_constraint[[f'oe_{impact}',f'oelf_{impact}',f'oeuf_{impact}',f'logp_{impact}']] = (
        pd.DataFrame(
            gnomad_constraint
                .apply(lambda x: poisson_test(x[f'obs_{impact}'], x[f'exp_{impact}']),axis=1)
            .tolist()
            ))

gnomad_constraint = gnomad_constraint[[
    'gene','gene_gnomad','transcript', 
    'obs_lof','exp_lof','oe_lof','oelf_lof','oeuf_lof', 'logp_lof',
    'obs_mis_pphen','exp_mis_pphen','oe_mis_pphen','oelf_mis_pphen','oeuf_mis_pphen', 'logp_mis_pphen',
    'obs_mis_non_pphen','exp_mis_non_pphen','oe_mis_non_pphen','oelf_mis_non_pphen','oeuf_mis_non_pphen', 'logp_mis_non_pphen',
    'obs_syn','exp_syn','oe_syn','oelf_syn','oeuf_syn', 'logp_syn',
    'pLI'
]]
gnomad_constraint.to_csv('../data/gpcr_genes_constraint_gnomad.tsv',sep='\t')

In [ ]:
# Add constraint from custom analysis
custom_constraint = pd.read_csv('../data/gpcr_genes_custom_constraint.tsv')
print(human_receptors.gene_gnomad.isna().sum())

human_receptors.merge(custom_constraint, on = 'gene_gnomad')
human_receptors.to_csv('../data/data/gpcr_genes_constraint_custom.txt')


# Essential genes from MacArthur lab

In [64]:
def clean_names(names):
    try:
        return eval(names)
    except:
        return names

gpcr_human_genes = pd.read_csv('../data/gpcr_genes_human_gpcrdb.tsv',sep='\t',converters={'gene_names': clean_names})
#gnomad_constraint = pd.read_csv(gnomad_constraint_path,sep='\t')

gene_name_lookup = gpcr_human_genes[['gene','gene_names']].explode(column='gene_names')
problematic_mappings = pd.DataFrame([
    ('ADRA1D', 'ADRA1A'),
    ('CCRL2', 'CCR6'),
    ('ACKR2','CCR10'),
    ('S1PR3','C9orf47'),
    ('PTGER4' ,'PTGER2'),
    ('FZD9' ,'FZD3')],columns =['gene','gene_names'])


with open('../data/gene_lists/lists/mgi_essential.tsv','r') as fid:
    mgi_essential_genes = [x.rstrip() for x in fid.readlines()]

gene_name_lookup['mouse_essential'] = gene_name_lookup.gene_names.isin(mgi_essential_genes)

with open('../data/gene_lists/lists/clinvar_path_likelypath.tsv','r') as fid:
    clinvar_disease_genes = [x.rstrip() for x in fid.readlines()]
gene_name_lookup['disease'] = gene_name_lookup.gene_names.isin(clinvar_disease_genes)

with open('../data/gene_lists/lists/gwascatalog.tsv','r') as fid:
    gwas_catalog_genes = [x.rstrip() for x in fid.readlines()]
gene_name_lookup['gwas'] = gene_name_lookup.gene_names.isin(gwas_catalog_genes)

genes_with_phenotypes = (
    gene_name_lookup.groupby('gene')
    .aggregate(np.any)
    .drop(columns = ['gene_names'])
    .reset_index()
)

curated_mgi_phenotypes = pd.read_csv('../data/gpcr_genes_mgi_essential_curated.tsv',sep='\t',index_col=0)
curated_mgi_phenotypes.columns = ['gene','curated_mouse_phenotype']
genes_with_phenotypes = genes_with_phenotypes.merge(curated_mgi_phenotypes,on='gene')

genes_with_phenotypes.to_csv('../data/gpcrs_with_phenotypes.tsv',sep='\t')
# print(gene_name_lookup.essential.sum())
# print(gene_name_lookup.disease.sum())
# print(gene_name_lookup.gwas.sum())

# print((gene_name_lookup.essential & gene_name_lookup.disease).sum())
# print((gene_name_lookup.essential | gene_name_lookup.disease).sum())
# print((gene_name_lookup.essential | gene_name_lookup.disease | gene_name_lookup.gwas).sum())

#gene_name_lookup


# Differences with curated version???

# Curated drug targets from GlobalData

* Check these for updates (FDA approvals) - none since 06/2022
* Look for adverse event indications of drugs using DrugBank?

In [54]:
gpcr_human_genes = pd.read_csv('../data/gpcr_genes_human_gpcrdb.tsv',sep='\t',converters={'gene_names': clean_names})
drug_targets = pd.read_csv('../data/gpcr_drug_targets.tsv',sep='\t')
gpcr_drug_targets = gpcr_human_genes.merge(drug_targets, left_on = 'entry_name',right_on ='target_uniprot',how='left')
gpcr_drug_targets = gpcr_drug_targets[['gene','entry_name','approved_drugs_moa','adverse_event_associations']]
gpcr_drug_targets['approved_drugs_moa'] = gpcr_drug_targets.approved_drugs_moa.fillna('none')
gpcr_drug_targets['adverse_event_associations'] = gpcr_drug_targets.adverse_event_associations.fillna('No')
gpcr_drug_targets.to_csv('../data/gpcrs_with_approved_drug_target_status.tsv',sep='\t')

# Make supplementary information

In [65]:
gpcrs_with_families = pd.read_csv('../data/gpcr_genes_human_gpcrdb.tsv',sep='\t')
gpcrs_with_constraint = pd.read_csv('../data/gpcr_genes_constraint_gnomad.tsv',sep='\t',index_col=0)
gpcrs_with_phenotypes = pd.read_csv('../data/gpcrs_with_phenotypes.tsv',sep='\t',index_col=0)
gpcrs_with_drug_targets = pd.read_csv('../data/gpcrs_with_approved_drug_target_status.tsv',sep='\t',index_col=0)

with pd.ExcelWriter('../data/Supplementary_Data.xlsx') as xls:
    gpcrs_with_families.to_excel(xls, sheet_name='Supplementary Table 1',index=False)
    gpcrs_with_constraint.to_excel(xls, sheet_name='Supplementary Table 2',index=False)
    gpcrs_with_phenotypes.to_excel(xls, sheet_name='Supplementary Table 3',index=False)
    gpcrs_with_drug_targets.to_excel(xls, sheet_name='Supplementary Table 4',index=False)